In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd

from geojson.geometry import Point, Polygon, MultiLineString


In [4]:
gdf_1 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_irradiation.pkl')
gdf_2 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_P.pkl')
gdf_3 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_rain.pkl')
gdf_4 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_RH.pkl')
gdf_5 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_temperature.pkl')
gdf_6 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_wind_dir.pkl')
gdf_7 = pd.read_pickle('./../cgarrido/madrid_data/weather/2022/gdf_proc_wind_speed.pkl')

gdf_1= gdf_1.rename(columns={"id": "station_id"})
gdf_2=gdf_2.rename(columns={"id": "station_id"})
gdf_3= gdf_3.rename(columns={"id": "station_id"})
gdf_4=gdf_4.rename(columns={"id": "station_id"})
gdf_5= gdf_5.rename(columns={"id": "station_id"})
gdf_6= gdf_6.rename(columns={"id": "station_id"})
gdf_7= gdf_7.rename(columns={"id": "station_id"})


In [5]:
gdf_list= [gdf_1, gdf_2, gdf_3, gdf_4, gdf_5, gdf_6, gdf_7 ]
gdf_property_list=["Irradiation", "P", "Rain", "RH", "Temperature", "Wind_Direction", "Wind_Speed"]
entity_id_prefix =  "urn:ngsi-ld:WeatherObserved:Madrid"
entity_type = 'WeatherObserved'


1- Check context and add context (smart data model link) to the @context tag

2- Geometry: Change geometry to location. Location is a geoproperty. Give the point values as "coordinates". Keep the observedat on every property.

3- Convert coordinates to lon/lat

4- Add no2, so2 as property (not as a separate entity). One entity per hexagon
. 
5- Add district instead of hexagons


In [ ]:


example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    sensor_name =gdf_property_list[property_index]
    counter =0
    uniqueStationIDs= tmp_gdf['station_id'].unique()
    dataset_length= len(tmp_gdf)
    feature_list= tmp_gdf.columns.tolist()

    for u in uniqueStationIDs:
        station_tmp_gdf = tmp_gdf[tmp_gdf['station_id'] == u] # take data coming only from this station ID (this sensor entity)
        entity_obj = Entity(entity_type, entity_id_prefix + sensor_name + ":ID:" + str(int(u)) ) # initialize the entity for the new station
        print(entity_id_prefix + str(sensor_name) + ":" + str(u))
        counter=0 # start pushing data about this entity
        for index, row in station_tmp_gdf.iterrows():

            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + sensor_name + ":ID:" + str(int(u))) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] == 'data':
                    entity_obj.add_prop(sensor_name, str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
                elif(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(station_tmp_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
    print("... published data for dataset: " +  sensor_name)

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


urn:ngsi-ld:WeatherObserved:MadridIrradiation:24
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:WeatherObserved:MadridIrradiation:ID:24', 'type': 'WeatherObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'station_id': {'type': 'Property', 'value': '24', 'observedAt': '2022-01-01T00:00:00Z'}, 'geometry': {'type': 'Property', 'value': 'POINT (-417152.48156697437 4927070.647350454)', 'observedAt': '2022-01-01T00:00:00Z'}, 'year': {'type': 'Property', 'value': '2022', 'observedAt': '2022-01-01T00:00:00Z'}, 'month': {'type': 'Property', 'value': '1', 'observedAt': '2022-01-01T00:00:00Z'}, 'day': {'type': 'Property', 'value': '1', 'observedAt': '2022-01-01T00:00:00Z'}, 'hour': {'type': 'Property', 'value': '0', 'observedAt': '2022-01-01T00:00:00Z'}, 'minute': {'type': 'Property', 'value': '0', 'observedAt': '2022-01-01T00:00:00Z'}, 'weekday': {'type': 'Property', 'value': '5', 'observedAt': '202

In [ ]:
gdf_1


# Query temporal entity from Scorpio and get it as Pandas

In [ ]:

client_obj = Client("localhost", "9090")
pandas_data= client_obj.get_temporal("urn:ngsi-ld:AirQualityObserved:Madrid:Hexagon:0", entityType="AirQualityObserved", pandasSeries=True, attribute="value")
print(pandas_data)
print(type(pandas_data))



## BACKUP CODE

import time


for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
        
    feature_list= tmp_gdf.columns.tolist()
    for index, row in tmp_gdf.iterrows():
        if index %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
            entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index]) # (re-)initialize the entity
        for i in range(len(feature_list)):
            if feature_list[i] =='type': feature_list[i] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
            if(feature_list[i] not in ['datetime','type']):
                entity_obj.add_prop(feature_list[i], str(row[i]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
        if index == 0:
            print('INFO: Initial row of the dataset to be published - as an example on how data looks.')
            print(entity_obj)
            
        #ingestion_starttime= time.perf_counter()
        temp_objects.append(entity_obj)
        if (index %num_rows_batch_publish==0 or index == len(tmp_gdf)-1) and  index!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
            publish(entity_obj.to_dict()) # publish the entity
    
        f(index%10000 ==0):
            print("... published data count: " +  str(index))

        #print('Info: Ingestion took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds for ' +  str(num_rows_batch_publish) + ' objects')

    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]))
